## 데이터 구성

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [37]:
# 트레인셋
train = pd.read_csv('train.csv', index_col=False,)
X_train = train.loc[:,'X00':'X39']
y_train = train.loc[:,'Y00':'Y17']

In [255]:
# 테스트 셋
test = pd.read_csv('test.csv', index_col=False,)
test=test.drop(['id'], axis=1)

## 모델링 1차
30일 간 데이터에 대한 모델 작성

In [12]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [7]:
# nan 값 제거
y = y_train.dropna()
y.shape

(4320, 18)

In [10]:
# 트레인 셋 범주 조정
X = X_train.loc[:y.shape[0]-1,:]
X.shape

(4320, 40)

In [14]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(X.shape[1], 1))) # 트레인값
model.add(Dense(y.shape[1])) # 출력값
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 18)                378       
Total params: 2,138
Trainable params: 2,138
Non-trainable params: 0
_________________________________________________________________


In [15]:
X = X.values
X = X.reshape(X.shape[0], X.shape[1], 1)

In [23]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model_1.fit(X, y, epochs=1000,
          batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/1000
4320/4320 [==============================] - ETA: 2s - loss: 6.129 - ETA: 2s - loss: 5.834 - ETA: 2s - loss: 6.429 - ETA: 2s - loss: 6.174 - ETA: 2s - loss: 6.290 - ETA: 2s - loss: 6.511 - ETA: 2s - loss: 6.523 - ETA: 2s - loss: 6.435 - ETA: 1s - loss: 6.605 - ETA: 1s - loss: 6.545 - ETA: 1s - loss: 6.723 - ETA: 1s - loss: 6.894 - ETA: 1s - loss: 6.957 - ETA: 1s - loss: 7.069 - ETA: 1s - loss: 7.074 - ETA: 1s - loss: 7.054 - ETA: 1s - loss: 7.050 - ETA: 1s - loss: 7.099 - ETA: 1s - loss: 7.165 - ETA: 1s - loss: 7.198 - ETA: 1s - loss: 7.192 - ETA: 1s - loss: 7.131 - ETA: 1s - loss: 7.124 - ETA: 1s - loss: 7.073 - ETA: 0s - loss: 7.074 - ETA: 0s - loss: 7.111 - ETA: 0s - loss: 7.071 - ETA: 0s - loss: 7.098 - ETA: 0s - loss: 7.100 - ETA: 0s - loss: 7.077 - ETA: 0s - loss: 7.047 - ETA: 0s - loss: 7.012 - ETA: 0s - loss: 6.992 - ETA: 0s - loss: 6.980 - ETA: 0s - loss: 6.970 - ETA: 0s - loss: 6.938 - ETA: 0s - loss: 6.913 - ETA: 0s - loss: 6.902 - ETA: 0s - loss: 6.921 - 2s 545

In [24]:
#모델 저장
import joblib
joblib.dump(model_1, 'model_1.pkl')

['model_1.pkl']

In [254]:
# 모델 적용 함수
def model_fit_1(data, model_name, file_name):
    model = joblib.load(str(model_name)) 
    data_r = data.values.reshape(data.shape[0],data.shape[1],1 )
    pred_out=model.predict(data_r)
    df = pd.DataFrame(pred_out)
    df.to_csv(str(file_name), index = False, header = False)
    return df

In [41]:
#지난 3일간의 데이터셋
X_test = X_train.loc[y.shape[0]:,:]

In [42]:
#지난3일간의 y값 연산
pred_out = model_fit_1(X_test, 'model_1.pkl', 'pred_out_last3dayy0017.csv')

## 모델링 2차
3일간의 데이터 y값과 x값 바탕으로 y18 연산

In [241]:
X=pd.concat([X_test.reset_index(drop=True), pred_out.reset_index(drop=True)], axis=1)

In [263]:
y = train['Y18']
y = y.dropna()

In [267]:
K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(X.shape[1], 1))) # (timestep, feature)
model.add(Dense(1)) # output = 1
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [268]:
X = X.values
X = X.reshape(X.shape[0], 58, 1)

In [269]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(X, y, epochs=1000,
          batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/1000
432/432 [==============================] - ETA: 6s - loss: 820.503 - ETA: 1s - loss: 781.329 - ETA: 0s - loss: 757.855 - ETA: 0s - loss: 758.613 - ETA: 0s - loss: 761.886 - 1s 2ms/step - loss: 763.3829
Epoch 2/1000
432/432 [==============================] - ETA: 0s - loss: 827.989 - ETA: 0s - loss: 759.091 - ETA: 0s - loss: 746.353 - ETA: 0s - loss: 741.330 - ETA: 0s - loss: 738.588 - 0s 776us/step - loss: 742.0738
Epoch 3/1000
432/432 [==============================] - ETA: 0s - loss: 748.272 - ETA: 0s - loss: 745.322 - ETA: 0s - loss: 728.012 - ETA: 0s - loss: 703.550 - ETA: 0s - loss: 711.573 - 0s 785us/step - loss: 721.1557
Epoch 4/1000
432/432 [==============================] - ETA: 0s - loss: 694.113 - ETA: 0s - loss: 746.601 - ETA: 0s - loss: 696.153 - ETA: 0s - loss: 696.022 - ETA: 0s - loss: 697.424 - 0s 771us/step - loss: 696.8721
Epoch 5/1000
432/432 [==============================] - ETA: 0s - loss: 666.209 - ETA: 0s - loss: 663.031 - ETA: 0s - loss: 615.311 - 

Epoch 35/1000
432/432 [==============================] - ETA: 0s - loss: 275.781 - ETA: 0s - loss: 285.285 - ETA: 0s - loss: 286.946 - ETA: 0s - loss: 279.454 - ETA: 0s - loss: 285.325 - ETA: 0s - loss: 286.004 - 0s 836us/step - loss: 286.6949
Epoch 36/1000
432/432 [==============================] - ETA: 0s - loss: 261.155 - ETA: 0s - loss: 260.269 - ETA: 0s - loss: 278.083 - ETA: 0s - loss: 270.727 - ETA: 0s - loss: 275.638 - ETA: 0s - loss: 278.957 - 0s 836us/step - loss: 280.7760
Epoch 37/1000
432/432 [==============================] - ETA: 0s - loss: 300.585 - ETA: 0s - loss: 280.159 - ETA: 0s - loss: 280.186 - ETA: 0s - loss: 280.393 - ETA: 0s - loss: 286.077 - ETA: 0s - loss: 284.690 - 0s 873us/step - loss: 275.0394
Epoch 38/1000
432/432 [==============================] - ETA: 0s - loss: 215.380 - ETA: 0s - loss: 242.633 - ETA: 0s - loss: 274.310 - ETA: 0s - loss: 257.428 - ETA: 0s - loss: 261.456 - ETA: 0s - loss: 270.302 - 0s 831us/step - loss: 269.3068
Epoch 39/1000
432/432 [=

432/432 [==============================] - ETA: 0s - loss: 62.13 - ETA: 0s - loss: 73.21 - ETA: 0s - loss: 79.88 - ETA: 0s - loss: 80.72 - ETA: 0s - loss: 81.82 - ETA: 0s - loss: 77.80 - ETA: 0s - loss: 79.10 - 0s 926us/step - loss: 80.1110
Epoch 99/1000
432/432 [==============================] - ETA: 0s - loss: 44.44 - ETA: 0s - loss: 80.08 - ETA: 0s - loss: 79.88 - ETA: 0s - loss: 81.81 - ETA: 0s - loss: 81.15 - ETA: 0s - loss: 80.33 - ETA: 0s - loss: 79.23 - 0s 919us/step - loss: 78.7112
Epoch 100/1000
432/432 [==============================] - ETA: 0s - loss: 115.228 - ETA: 0s - loss: 91.166 - ETA: 0s - loss: 81.59 - ETA: 0s - loss: 76.22 - ETA: 0s - loss: 76.80 - 0s 787us/step - loss: 77.3394
Epoch 101/1000
432/432 [==============================] - ETA: 0s - loss: 68.03 - ETA: 0s - loss: 61.08 - ETA: 0s - loss: 66.92 - ETA: 0s - loss: 71.79 - ETA: 0s - loss: 77.56 - 0s 790us/step - loss: 76.0136
Epoch 102/1000
432/432 [==============================] - ETA: 0s - loss: 47.18 - ETA

Epoch 133/1000
432/432 [==============================] - ETA: 0s - loss: 31.88 - ETA: 0s - loss: 55.53 - ETA: 0s - loss: 55.14 - ETA: 0s - loss: 48.45 - ETA: 0s - loss: 47.91 - ETA: 0s - loss: 51.48 - 0s 875us/step - loss: 49.5165
Epoch 134/1000
432/432 [==============================] - ETA: 0s - loss: 58.99 - ETA: 0s - loss: 46.37 - ETA: 0s - loss: 50.19 - ETA: 0s - loss: 48.79 - ETA: 0s - loss: 48.77 - ETA: 0s - loss: 48.94 - 0s 866us/step - loss: 49.0685
Epoch 135/1000
432/432 [==============================] - ETA: 0s - loss: 32.80 - ETA: 0s - loss: 41.80 - ETA: 0s - loss: 47.18 - ETA: 0s - loss: 47.36 - ETA: 0s - loss: 49.53 - ETA: 0s - loss: 46.74 - 0s 847us/step - loss: 48.6375
Epoch 136/1000
432/432 [==============================] - ETA: 0s - loss: 34.61 - ETA: 0s - loss: 43.24 - ETA: 0s - loss: 41.74 - ETA: 0s - loss: 44.49 - ETA: 0s - loss: 45.99 - ETA: 0s - loss: 46.52 - ETA: 0s - loss: 45.38 - ETA: 0s - loss: 47.44 - 1s 1ms/step - loss: 48.2105
Epoch 137/1000
432/432 [==

Epoch 196/1000
432/432 [==============================] - ETA: 0s - loss: 19.06 - ETA: 0s - loss: 21.32 - ETA: 0s - loss: 21.79 - ETA: 0s - loss: 18.73 - ETA: 0s - loss: 19.49 - ETA: 0s - loss: 19.04 - 0s 993us/step - loss: 19.1574
Epoch 197/1000
432/432 [==============================] - ETA: 0s - loss: 10.47 - ETA: 0s - loss: 12.39 - ETA: 0s - loss: 17.77 - ETA: 0s - loss: 18.05 - ETA: 0s - loss: 19.13 - ETA: 0s - loss: 17.93 - 0s 1ms/step - loss: 18.8591
Epoch 198/1000
432/432 [==============================] - ETA: 0s - loss: 28.06 - ETA: 0s - loss: 18.91 - ETA: 0s - loss: 16.34 - ETA: 0s - loss: 19.35 - ETA: 0s - loss: 18.40 - 0s 769us/step - loss: 18.5604
Epoch 199/1000
432/432 [==============================] - ETA: 0s - loss: 22.79 - ETA: 0s - loss: 15.94 - ETA: 0s - loss: 17.04 - ETA: 0s - loss: 17.47 - ETA: 0s - loss: 18.81 - 0s 806us/step - loss: 18.2766
Epoch 200/1000
432/432 [==============================] - ETA: 0s - loss: 28.92 - ETA: 0s - loss: 22.24 - ETA: 0s - loss: 

In [270]:
joblib.dump(model, 'model_2.pkl')

['model_2.pkl']

In [271]:
# 모델2 적용 함수
def model_fit_2(data1, data2, model_name, file_name):
    data=pd.concat([data1.reset_index(drop=True), data2.reset_index(drop=True)], axis=1)
    model = joblib.load(str(model_name))
    data_r = data.values.reshape(data.shape[0],data.shape[1],1 )
    pred_out=model.predict(data_r)
    df = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':pred_out.reshape(1,-1)[0]})
    df.to_csv(file_name, index = False)
    return df

## 테스트

In [272]:
pred_out_1 = model_fit_1(test, 'model_1.pkl', 'pred_out_1.csv')

In [273]:
pred_out_fin = model_fit_2(test, pred_out_1, 'model_2.pkl', 'pred_out_fin.csv')